# ELECTRA를 이용한 한국어 감정분류
사전학습모델 : [KcELECTRA](https://github.com/Beomi/KcELECTRA) <br>
데이터 : [NAVER Sentiment Movie Corpus](https://github.com/e9t/nsmc/)

# 사전 준비

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 7.8 MB/s 
     |████████████████████████████████| 163 kB 68.3 MB/s 
     |████████████████████████████████| 7.6 MB 44.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 8.2 MB/s 
     |████████████████████████████████| 115 kB 65.4 MB/s 
     |████████████████████████████████| 212 kB 67.1 MB/s 
     |████████████████████████████████| 127 kB 68.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import warnings

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

**NSMC 데이터 불러오기**

In [3]:
from datasets import load_dataset

datasets = load_dataset("nsmc")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [5]:
# label 0: negative(부정) / 1: positive(긍정)
for i in range(3):
    print("train", datasets["train"][i])
    print("test", datasets["test"][i])

train {'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}
test {'id': '6270596', 'document': '굳 ㅋ', 'label': 1}
train {'id': '3819312', 'document': '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'label': 1}
test {'id': '9274899', 'document': 'GDNTOPCLASSINTHECLUB', 'label': 0}
train {'id': '10265843', 'document': '너무재밓었다그래서보는것을추천한다', 'label': 0}
test {'id': '8544678', 'document': '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', 'label': 0}


**KcELECTRA 모델과 토크나이저 불러오기**

In [6]:
from transformers import AutoTokenizer, AutoModel
  
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
model = AutoModel.from_pretrained("beomi/KcELECTRA-base")

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
model.config

ElectraConfig {
  "_name_or_path": "beomi/KcELECTRA-base",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50135
}

# 데이터 구축

**데이터 준비**

In [8]:
from tqdm.auto import tqdm as tqdm_auto

In [9]:
# 150000개 처리 시간 약 8시간 -> 따라서 train:test=3:1 유지시키고 전체 개수를 1/15으로 줄임
ids = int((datasets['train'].num_rows)//15)
train_doc = [datasets['train']['document'][idx] for idx in tqdm_auto(range(0, ids))]
train_label = [datasets['train']['label'][idx] for idx in tqdm_auto(range(0, ids))]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [10]:
ids = int((datasets['test'].num_rows)//15)
test_doc = [datasets['test']['document'][idx] for idx in tqdm_auto(range(0, ids))]
test_label = [datasets['test']['label'][idx] for idx in tqdm_auto(range(0, ids))]

  0%|          | 0/3333 [00:00<?, ?it/s]

  0%|          | 0/3333 [00:00<?, ?it/s]

**텍스트 전처리** <br>
Pre-trained KcELECTRA는 preprocessing 과정이 존재 / 해당 저자의 finetune 모델에선 사용하지 않는다.

In [11]:
!pip install soynlp
!pip install emoji==1.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 7.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 7.6 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=cf1ca9233feb97c4d98de41b98a13f58ec5cf50b9632271e04868b3abd4e0c92
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [12]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

In [13]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

In [14]:
def clean(x):
    x = pattern.sub(' ', x)         # 일반적으로 사용하는 특수문자, 영어, 한글, emoji제외 공백으로 치환
    x = url_pattern.sub('', x)      # URL 제거
    x = x.strip()                   # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    return x

In [15]:
train_clean = [clean(train_doc[idx]) for idx in range(0, len(train_doc))]
test_clean = [clean(test_doc[idx]) for idx in range(0, len(test_doc))]

In [16]:
print("전처리 전:", train_doc[10])
print("전처리 후:", train_clean[10])

전처리 전: 걍인피니트가짱이다.진짜짱이다♥
전처리 후: 걍인피니트가짱이다.진짜짱이다


**토크나이징**

In [17]:
# KcELECTRA모델에서 정한 입력 크기보다 크면 잘라내기, 패딩 채우기
train_input = tokenizer(train_clean, truncation=True, padding=True, return_tensors="pt")
test_input = tokenizer(test_clean, truncation=True, padding=True, return_tensors="pt")

**데이터셋 변환**

In [18]:
import torch

In [19]:
class NSMCDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [20]:
train_dataset = NSMCDataset(train_input, train_label)
test_dataset = NSMCDataset(test_input, test_label)

In [21]:
for n in range(3):
    print("train_dataset[",n,"]")
    print(train_dataset[n])

train_dataset[ 0 ]
{'input_ids': tensor([    2,  2431, 48502,    18,    18,  7997, 15775,  8028, 10855,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

**데이터로더 정의**

In [22]:
from torch.utils.data import DataLoader

In [23]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [24]:
# 데이터로더 확인
next(iter(train_loader))

{'input_ids': tensor([[    2,  8022, 32561,  ...,     0,     0,     0],
         [    2, 29800,  1399,  ...,     0,     0,     0],
         [    2,    25,  4301,  ...,     0,     0,     0],
         ...,
         [    2,  7997, 10431,  ...,     0,     0,     0],
         [    2,  8098, 16025,  ...,     0,     0,     0],
         [    2, 31028,  4192,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1])}

# 모델 학습

**모델 정의**

In [25]:
import torch.nn as nn

In [26]:
# ELECTRA를 포함한 신경망 모형
class ELECTRA_Model(torch.nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels): 
        super(ELECTRA_Model, self).__init__()
        self.token_size = token_size
        self.num_labels = num_labels
        self.pretrained_model = pretrained_model

        # 분류기 정의
        self.classifier = torch.nn.Linear(self.token_size, self.num_labels)

    def forward(self, inputs):
        # 모형에 입력을 넣고 출력을 받음
        outputs = self.pretrained_model(**inputs)
        # 출력에서 CLS 토큰에 해당하는 부분만 가져옴
        clf_token = outputs.last_hidden_state[:,0,:]
        
        return self.classifier(clf_token)

electra = ELECTRA_Model(model, num_labels=2, token_size=model.config.hidden_size)

**모델 파라미터 설정**

In [27]:
from transformers import get_linear_schedule_with_warmup

In [28]:
# GPU 가속을 사용할 수 있으면 device를 cuda로 설정하고, 아니면 cpu로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 5
learning_rate = 1e-5

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# 학습 스케줄러 설정
scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                            num_training_steps=len(train_loader),
                                            num_warmup_steps=0)

**학습 진행**

In [29]:
electra.to(device)
electra.train()

ELECTRA_Model(
  (pretrained_model): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

In [30]:
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [31]:
for epoch in range(num_epochs):
    loss = 0
    train_loss = 0.0
    n = 0
    train_accuracy = 0
        
    for batch in tqdm_auto(train_loader, mininterval=0.01, leave=True):
        optimizer.zero_grad()     # 그래디언트 초기화

        # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
        labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
        outputs = electra(inputs)   # 모형으로 결과 예측

        loss = criterion(outputs, F.one_hot(labels, num_classes=2).float())
        train_loss += loss
        
        loss.backward()
        optimizer.step()
        scheduler.step()
            
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy += accuracy_score(labels.cpu(), outputs.argmax(dim=1).cpu())
        n += 1

    train_accuracy /= n

    print('epoch %d, train loss: %.4f, accuracy_score: %.2f \n' % (epoch, avg_train_loss, train_accuracy))

  0%|          | 0/625 [00:00<?, ?it/s]

epoch 0, train loss: 0.3556, accuracy_score: 0.84 



  0%|          | 0/625 [00:00<?, ?it/s]

epoch 1, train loss: 0.2582, accuracy_score: 0.90 



  0%|          | 0/625 [00:00<?, ?it/s]

epoch 2, train loss: 0.2570, accuracy_score: 0.90 



  0%|          | 0/625 [00:00<?, ?it/s]

epoch 3, train loss: 0.2586, accuracy_score: 0.90 



  0%|          | 0/625 [00:00<?, ?it/s]

epoch 4, train loss: 0.2561, accuracy_score: 0.90 



# 모델 테스트

In [32]:
electra.eval()
test_accuracy = 0
n = 0

for batch in test_loader:
    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
    labels = batch['labels'].to(device)
    
    with torch.no_grad():   # 학습 X (그래디언트 계산 X)
        outputs = electra(inputs)

    test_accuracy += accuracy_score(labels.cpu(), outputs.argmax(dim=1).cpu())
    n += 1

test_accuracy /= n

print('test accuracy_score: %.2f \n' % (test_accuracy))

test accuracy_score: 0.90 

